#### Let's validate the model using DRUM

In [1]:
!drum validation --code-dir ../src/custom_model --input ../data/boston_housing_test.csv --target-type regression > drum_validation.log

Traceback (most recent call last):
  File "/Users/timothy.whittaker/python_envs/odsc/bin/drum", line 6, in <module>
    main()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/main.py", line 96, in main
    CMRunner(runtime).run()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/drum.py", line 289, in run
    self._run_fit_and_predictions_pipelines_in_mlpiper()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/drum.py", line 541, in _run_fit_and_predictions_pipelines_in_mlpiper
    _pipeline_executor.init_pipeline()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/mlpiper/pipeline/executor.py", line 257, in init_pipeline
    self._init_pipeline()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/mlpiper/pipeline/executor.py", line 204, in _init_pipeline
    self._init_ml_engine(self.pip

In [6]:
!tail -20 drum_validation.log

19       17.690


Validation checks results
      Test case         Status                       Details                    
Null value imputation   FAILED   Null value imputation check performs check by  
                                 imputing each feature with NaN value. If check 
                                 fails for a feature, test dataset is saved in  
                                 /tmp/drum_validation_checks_. Make sure to     
                                 delete those folders if it takes too much      
                                 space.                                         
                                                                                
                                 Failed feature                          Dataset
                                 filename                                       
                                 lstat            /tmp/drum_validation_checks_xk
                                 6ivf_v/null_value_imputation_lst

As you can see, we are getting an error that `ValueError: Failure when making predictions: Input contains NaN, infinity or a value too large for dtype('float32').`

Basically, the model is not robust enough to handle null values and while our data does not include any null values, DRUM is checking automatically. To solve this, we are going to add a `custom.py` script within the `custom_model` folder which will fill null values if they are encountered.

When models don't fit the drop in pattern, we can create custom hooks to hook the model up to drum.

In [10]:
!head -10 ../src/custom.py

import pandas as pd
from typing import List, Optional, Any, Dict

# """
# Custom hooks for prediction
# ---------------------------
# If drum's standard assumptions are incorrect for your model, **drum** supports several hooks
# for custom inference code.
# """
# def init(code_dir : Optional[str], **kwargs) -> None:


In the code above, we have only created a transform hook which could handle preprocessing including missing value fill in.  

In [8]:
!cp ../src/custom.py ../src/custom_model/custom.py

In [9]:
!drum validation --code-dir ../src/custom_model --input ../data/boston_housing_test.csv --target-type regression > drum_validation_2.log

Traceback (most recent call last):
  File "/Users/timothy.whittaker/python_envs/odsc/bin/drum", line 6, in <module>
    main()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/main.py", line 96, in main
    CMRunner(runtime).run()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/drum.py", line 289, in run
    self._run_fit_and_predictions_pipelines_in_mlpiper()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/drum.py", line 541, in _run_fit_and_predictions_pipelines_in_mlpiper
    _pipeline_executor.init_pipeline()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/mlpiper/pipeline/executor.py", line 257, in init_pipeline
    self._init_pipeline()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/mlpiper/pipeline/executor.py", line 204, in _init_pipeline
    self._init_ml_engine(self.pip

In [11]:
!tail -20 drum_validation_2.log

19       17.690


Validation checks results
      Test case         Status                       Details                    
Null value imputation   FAILED   Null value imputation check performs check by  
                                 imputing each feature with NaN value. If check 
                                 fails for a feature, test dataset is saved in  
                                 /tmp/drum_validation_checks_. Make sure to     
                                 delete those folders if it takes too much      
                                 space.                                         
                                                                                
                                 Failed feature                          Dataset
                                 filename                                       
                                 lstat            /tmp/drum_validation_checks_ws
                                 r78kpv/null_value_imputation_lst

After moving the `custom.py` file inside the `custom_model` folder, we see that the results come back normally.